In [1]:
# import all the necessaries librairies
import pulp as p
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
import pulp as p

solver = p.GLPK_CMD(path='/Users/charlesbacquaert/anaconda3/envs/Projet_Corners_Outil_Carte/bin/glpsol')

#solver = p.GLPK()
print(solver.available())

solver_list = p.listSolvers(onlyAvailable=True)
print(solver_list)

/Users/charlesbacquaert/anaconda3/envs/Projet_Corners_Outil_Carte/bin/glpsol
['PULP_CBC_CMD']


In [3]:
# connect to google sheet with the json key
gc = gspread.service_account(filename='projetcorneroutilcarte-8eff842e6082.json')

# connect Sheets of EPD available and constraints
doc_outil_carte = gc.open_by_url("https://docs.google.com/spreadsheets/d/1wGohQGY_qV9f1dyeNQm9n6S7PVnNnTCC-QKybBPsw5U/")
sheet_outil_carte = doc_outil_carte.worksheet("Input")
sheet_contraintes_jours = doc_outil_carte.worksheet("Contraintes Jours")
sheet_contraintes_sous_categories = doc_outil_carte.worksheet("Contraintes Sous Categories")
sheet_contraintes_particulieres = doc_outil_carte.worksheet("Contraintes Particulieres")

data = sheet_outil_carte.get('A2:K')
headers = data.pop(0)

# transform the data from google sheet to data frame that Python can read and display the first 15 rows of data
df = pd.DataFrame(data, columns=headers)


In [4]:
# Define input data
#print(df.columns)

# useful, data in Input



# useless, data in Input 
df["ID"].replace('', np.nan, inplace=True)
df.dropna(subset=["ID"], inplace=True)
df["ID"] = df["ID"].astype(int)
df["Fournisseur"]=df["Fournisseur"].astype(str)
df["EPD"]=df["EPD"].astype(str)
df["Sous Categorie Corner"]=df["Sous Categorie Corner"].astype(str)
df["FC"] = df["FC"].astype(float)
df["PP semaine"]=df["PP semaine"].astype(float)
df["PP week-end"]=df["PP week-end"].astype(float)
df["DLC"] = df["DLC"].astype(int)
df["AA"] = df["AA"].astype(int)
df[" Prix"] = df[" Prix"].astype(float)


# useless, no data in Input
#df["AA lundi"] = df["AA lundi"].astype(int)
#df["Province"] = df["Province"].astype(int)
#df["Regime"] = df["Regime"].astype(str)
#df["Sous Categ Proteine"] = df["Sous Categ Proteine"].astype(str)
#df["Forme légumes"] = df["Forme légumes"].astype(str)
#df["Soupe equivalent"] = df["Soupe equivalent"].astype(str)
#df["Base"] = df["Base"].astype(str)
#df["Chaud/Froid"] = df["Chaud/Froid"].astype(str)
#df["Chocolat"] = df["Chocolat"].astype(str)
#df["F1"] = df["F1"].astype(int)
#df["F2"] = df["F2"].astype(int)
#df["F3"] = df["F3"].astype(int)
#df["R"] = df["R"].astype(int)
#df["R Province"] = df["R Province"].astype(int)
#df["Carte Dimanche"] = df["Carte Dimanche"].astype(int)
#df["Fromage blanc_yaourt"] = df["Fromage blanc_yaourt"].astype(int)

In [5]:
# Check if "EPD" is in the list of column names
print(df.columns)

Index(['ID', 'Titre', 'Fournisseur', 'EPD', 'Sous Categorie Corner', 'FC',
       'PP semaine', 'PP week-end', 'DLC', ' Prix', 'AA'],
      dtype='object')


In [6]:
nb_corners = int(sheet_contraintes_jours.get('B2')[0][0])
#print(nb_corners)
corners = []

for c in range(nb_corners):
    #print(sheet_contraintes_jours.get(f'B{5+i}'))
    corners.append(sheet_contraintes_jours.get(f'B{5+c}')[0][0])
print(corners)

EPD = ["ENTREE","PLAT","DESSERT"]
JOURS = ["Lundi","Mardi","Mercredi","Jeudi","Vendredi","Samedi","Dimanche"]

['Inuï', 'Kinn Khao', 'Neko Ramen', 'Pastavino', 'Bunavino', 'Shaki Shaki', 'Zohra', 'Janet', 'Bao Family', 'Yann Couvreur', "La Saveur d'Abord"]


In [7]:
# Define the optimization problem
prob = p.LpProblem("Outil_Carte", p.LpMinimize)

# Define the decision variables
carte_vars = p.LpVariable.dicts("Carte", [(i, j) for i in df.index for j in range(7)], cat="Binary")
#dish_count_vars = p.LpVariable.dicts("Dish_Count", df.index, cat="Integer", lowBound=0)

# Define the binary varimathaables y_i
#y_vars = p.LpVariable.dicts("y", df.index, cat=p.LpBinary)

In [8]:
food_cost_expr = - p.lpSum(carte_vars[(i, j)] for i in df.index for j in range(7))
objective_expr = food_cost_expr
prob += objective_expr

In [9]:
for index_corner in range(nb_corners):
    corner = corners[index_corner]
    print(corner)
    prod_max_corner = sheet_contraintes_jours.range(f'C{5+index_corner}:W{5+index_corner}')
    #print(prod_max_corner)
    for index_jour in range(len(JOURS)):
        prod_max_corner_jour = prod_max_corner[index_jour*3:(index_jour+1)*3]
        #print(prod_max_corner_jour)
        for index_EPD in range(len(EPD)):
            prod_max_corner_jour_typeEPD = 0 if prod_max_corner_jour[index_EPD].value == "" else int(prod_max_corner_jour[index_EPD].value)
            #print(prod_max_corner_jour_typeEPD)
            prob += p.lpSum(carte_vars[(produit, index_jour)] for produit in df.index if df.loc[produit, "EPD"] == EPD[index_EPD] and df.loc[produit, "Fournisseur"] == corner ) <= prod_max_corner_jour_typeEPD
            #print(corner,JOURS[index_jour],EPD[index_EPD])
            #for produit in df.index:
                #if df.loc[produit, "EPD"] == EPD[index_EPD] and df.loc[produit, "Fournisseur"] == corner:
                    #print(df.loc[produit,"Titre"])
                    #print(f"Valeur de carte_vars[{produit}, {index_jour}]: {carte_vars[produit, index_jour].varValue}")
            
            
            

Inuï
Kinn Khao
Neko Ramen
Pastavino
Bunavino
Shaki Shaki
Zohra
Janet
Bao Family
Yann Couvreur
La Saveur d'Abord


In [10]:
# Solve the problem
prob.solve(solver)

# Check the status of the problem
status = p.LpStatus[prob.status]
print(status)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --cpxlp /var/folders/9z/v02bywcx5mxgvpjh2q8zvhk40000gn/T/7e806fee3827437f9e3dfcf40c87b6a5-pulp.lp
 -o /var/folders/9z/v02bywcx5mxgvpjh2q8zvhk40000gn/T/7e806fee3827437f9e3dfcf40c87b6a5-pulp.sol
Reading problem data from '/var/folders/9z/v02bywcx5mxgvpjh2q8zvhk40000gn/T/7e806fee3827437f9e3dfcf40c87b6a5-pulp.lp'...
232 rows, 680 columns, 799 non-zeros
679 integer variables, all of which are binary
1182 lines were read
GLPK Integer Optimizer, v4.65
232 rows, 680 columns, 799 non-zeros
679 integer variables, all of which are binary
Preprocessing...
71 rows, 543 columns, 543 non-zeros
543 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 71
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
71 rows, 543 columns, 543 non-zeros
*     0: obj =  -3.20

In [18]:
# Print the optimal total food cost
print(f"Objective function: {p.value(prob.objective):,.2f}")

# Print the optimal menu for each day
result_list = [] 
for j in range(7):
    for i in df.index:
        if carte_vars[(i, j)].varValue == 1:
            result_dict = {"Day of the week " : j+1,
                          "ID" : df.loc[i, 'ID'],
                           "Titre" : df.loc[i, 'Titre'],
                           "Fournisseur" : df.loc[i,"Fournisseur"],
                           "EPD" : df.loc[i,"EPD"],
                           "Sous Categorie Corner" : df.loc[i,"Sous Categorie Corner"],
                           "DLC" : df.loc[i, "DLC"]
                          }
            result_list.append(result_dict)

df_result = pd.concat([pd.DataFrame(dict_ID, index=[0]) for dict_ID in result_list], ignore_index=True)

# Print the value of each expression
print("Total food cost: ", p.value(food_cost_expr))

for i in df.index:
    for j in range(7):
        print(f"Valeur de carte_vars[{i}, {j}]: {carte_vars[i, j].varValue}")


Objective function: -186.00
Total food cost:  -186
Valeur de carte_vars[0, 0]: 1
Valeur de carte_vars[0, 1]: 1
Valeur de carte_vars[0, 2]: 1
Valeur de carte_vars[0, 3]: 1
Valeur de carte_vars[0, 4]: 1
Valeur de carte_vars[0, 5]: 1
Valeur de carte_vars[0, 6]: 0
Valeur de carte_vars[1, 0]: 1
Valeur de carte_vars[1, 1]: 1
Valeur de carte_vars[1, 2]: 1
Valeur de carte_vars[1, 3]: 1
Valeur de carte_vars[1, 4]: 1
Valeur de carte_vars[1, 5]: 1
Valeur de carte_vars[1, 6]: 0
Valeur de carte_vars[2, 0]: 1
Valeur de carte_vars[2, 1]: 1
Valeur de carte_vars[2, 2]: 1
Valeur de carte_vars[2, 3]: 1
Valeur de carte_vars[2, 4]: 1
Valeur de carte_vars[2, 5]: 1
Valeur de carte_vars[2, 6]: 0
Valeur de carte_vars[3, 0]: 0
Valeur de carte_vars[3, 1]: 0
Valeur de carte_vars[3, 2]: 0
Valeur de carte_vars[3, 3]: 0
Valeur de carte_vars[3, 4]: 0
Valeur de carte_vars[3, 5]: 0
Valeur de carte_vars[3, 6]: 0
Valeur de carte_vars[4, 0]: 0
Valeur de carte_vars[4, 1]: 0
Valeur de carte_vars[4, 2]: 0
Valeur de carte_var

In [19]:
# Write the result back to Google sheets starting from the second row
sheet_output = doc_outil_carte.worksheet("Output")
#Delete the old output
doc_outil_carte.values_clear("Output!C1:Q500")

#Paste the new output
sheet_output.update('C1',[df_result.columns.values.tolist()] + df_result.values.tolist())
sheet_output.update('A2', f"{p.value(food_cost_expr):,.2f}")

{'spreadsheetId': '1wGohQGY_qV9f1dyeNQm9n6S7PVnNnTCC-QKybBPsw5U',
 'updatedRange': 'Output!A2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [12]:
nb_sous_categories = int(sheet_contraintes_sous_categories.get('A2')[0][0])
print(nb_sous_categories)

36
